# **Introducción a los Servicios Fundamentales de AWS e implementación de seguridad**

# Introducción

En este notebook, exploraremos cuatro servicios fundamentales de AWS:

1. **IAM (Identity and Access Management)** 🛡️
2. **AWS Networking: VPC, Subnets, NAT, Internet Gateway, Route Tables, ACL** 🌐
3. **Máquinas Virtuales (EC2)** 💻
4. **Contenedores (ECS)** 📦
5. **Computación Serverless (Lambda)** ⚡

Veremos las características de cada uno y cómo se implementan en AWS mediante este ejercicio.

## Diagrama de ejercicio general

![Diagram](assets/diagram.png)

## Explicación Arquitectura en AWS 

En este ejercicio vamos a crear una arquitectura en AWS que utiliza una VPC con subnets públicas y privadas:

- **Subnet pública**: alojará una aplicación web. 🌐
- **Subnet privada**: contendrá una base de datos. 💾

## Componentes clave:

- **Internet Gateway**: permitirá el acceso externo. 🌍
- **NAT Gateway**: permitirá que la base de datos acceda a internet sin ser expuesta. 🔒
- **Security Groups**: protegerán los recursos controlando el tráfico. 🛡️
- **ECR (Elastic Container Registry)**: almacenará imágenes de contenedores utilizadas por la aplicación. 📦
- **Lambda**: se empleará para hacer llamadas al servicio web. ⚡
- **S3**: almacenará los logs de las llamadas Lambda. La Lambda tendrá un rol con acceso exclusivo a un bucket S3. 🗃️
- **Session Manager**: se usará para una conexión segura a las instancias. 🔐
- **Tablas de enrutamiento**: gestionarán el flujo del tráfico dentro de la VPC. 🛣️

Esta arquitectura asegura una comunicación eficiente y segura entre los diferentes componentes.


# IAM (Identity and Access Management)

**IAM** es un servicio web que ayuda a controlar de forma segura el acceso a los recursos de AWS. 🛡️

Características:
- Gestión centralizada de usuarios y accesos 👥
- Autenticación y autorización granular 🔐
- Integración con servicios AWS 🔗
- Soporte para identidades federadas 🌍
- Sin costo adicional 💸


## Conceptos Clave

- **Cuenta Root**: Al crear una cuenta de AWS, se crea una cuenta root por defecto. Esta cuenta tiene acceso completo a todos los recursos y servicios de AWS y **no debe ser compartida**. 🔑
- **Usuarios**: Son entidades que creas en AWS para representar personas o servicios que interactúan con los recursos de AWS. Cada usuario tiene un conjunto único de credenciales. 👤
- **Grupos**: Son colecciones de usuarios. No pueden contener otros grupos, solo usuarios. Los usuarios pueden pertenecer a uno o más grupos. 👥
- **Roles**: Son similares a los usuarios, pero están destinados a ser asumidos por cualquier entidad que necesite acceso temporal a recursos específicos. 🎭
- **Políticas**: Definen permisos y pueden ser adjuntadas a usuarios, grupos o roles. 📜


### 🚀 Ejemplo: Asignar nuevo usuario a grupo con acceso a S3 🛠️

En este ejercicio, vamos a crear un **usuario** y un **grupo** en AWS. El grupo tendrá permisos exclusivamente para **enlistar** y **crear buckets** en S3. 📝🔒

Pasos a seguir:

1. **Crear un usuario en IAM** 👤
2. **Crear un grupo en IAM** 👥
3. **Asignar permisos al grupo** 🛡️
4. **Añadir el usuario al grupo** 🔗


In [6]:
# Importamos la función IFrame del módulo IPython.display
from IPython.display import IFrame

# Definimos la ruta del archivo PDF
pdf_path = 'assets/iam.pdf'

# 📝🖇️ Información sobre el PDF
print("Título: IAM (Identity and Access Management)")
print("📁 Ruta del archivo PDF: 'assets/iam.pdf'")
print("📑🖥️ Mostrando el archivo PDF...")

# Usamos IFrame para mostrar el PDF
IFrame(pdf_path, width=1000, height=500)



Título: IAM (Identity and Access Management)
📁 Ruta del archivo PDF: 'assets/iam.pdf'
📑🖥️ Mostrando el archivo PDF...


### 🚀 Ejemplo: Crear un Rol de Administrador y Asumirlo por el Usuario 🛠️

En este ejercicio, crearemos un **rol** de tipo Administrador, que tendrá permisos para **crear** y **leer** cualquier recurso en la cuenta de AWS. Además, otorgaremos al **usuario** los permisos necesarios para asumir este rol y ejecutar dichas acciones. 👤🔄👑

Pasos a seguir:

1. **Crear un rol de Administrador en IAM** 👑
2. **Definir la política de asunción del rol** 📜
3. **Asignar permisos al rol** 🛡️
4. **Otorgar al usuario permisos para asumir el rol** 🔗
5. **Asumir el rol como usuario** 🔄

In [7]:
# Importamos la función IFrame del módulo IPython.display
from IPython.display import IFrame

# Definimos la ruta del archivo PDF
pdf_path = 'assets/roleassume.pdf'

# 📝🖇️ Información sobre el PDF
print("Título: Creación de Rol de Administrador y Asumirlo")
print("📁 Ruta del archivo PDF: 'assets/roleassume.pdf'")
print("📑🖥️ Mostrando el archivo PDF...")

# Usamos IFrame para mostrar el PDF
IFrame(pdf_path, width=1000, height=500)

Título: Creación de Rol de Administrador y Asumirlo
📁 Ruta del archivo PDF: 'assets/roleassume.pdf'
📑🖥️ Mostrando el archivo PDF...


### 📚 Documentación  📄🔍

Consultar esta información para más detalles:

- [📝 Crear usuarios en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_users_create.html)
- [👥 Crear grupos en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_groups_create.html)
- [🔧 Crear roles para servicios en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles_create_for-service.html)
- [🏷️ Crear roles en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles_create.html)
- [🎭 Roles de IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles.html)
- [🔄 Asumir roles de AWS](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles_use.html)

# Setup Inicial para ejecutar proyecto 🚀

### Importar bibliotecas 🚀

In [8]:
import boto3
from botocore.exceptions import ClientError
import json
import time
import requests

### Configurar el cliente de AWS (asegúrate de tener tus credenciales configuradas) 🚀

In [9]:
iam = boto3.client('iam')
ec2 = boto3.client('ec2')
lambda_client = boto3.client('lambda')
ecs = boto3.client('ecs')

# AWS Networking: VPC, Subnets, NAT, Internet Gateway, Route Tables, ACL 🌐

AWS Networking se centra en la configuración y gestión de redes dentro de AWS utilizando varios componentes clave:

- **VPC (Virtual Private Cloud):** Una red virtual aislada dentro de AWS donde se despliegan recursos. 🏠
- **Subnets:** Segmentos de una VPC que permiten organizar y asegurar los recursos de manera granular. 📊
- **NAT (Network Address Translation):** Permite que instancias en subnets privadas accedan a Internet sin exponer sus direcciones IP privadas. 🔄
- **Internet Gateway:** Un componente que permite la comunicación entre instancias en una VPC y la Internet. 🌍
- **Route Tables:** Definen las reglas de enrutamiento para controlar el tráfico dentro de la VPC y hacia fuera. 🛣️
- **ACL (Access Control Lists):** Listas de control de acceso que proporcionan una capa de seguridad adicional a nivel de subred, permitiendo o denegando tráfico. 🔒

Estos componentes juntos permiten diseñar y gestionar redes seguras y eficientes en la nube de AWS.


## Creación y Configuración de una VPC 🏗️

Una Virtual Private Cloud (VPC) es una red virtual en la nube de AWS, aislada lógicamente de otras redes virtuales. A continuación, se detalla cómo crear y configurar una VPC usando Boto3, la biblioteca de AWS para Python.

In [10]:
print("🚀 Iniciando la creación de la VPC...")

# Crear una VPC
response = ec2.create_vpc(
    CidrBlock='10.0.0.0/16'
)

vpc_id = response['Vpc']['VpcId']
print(f'✅ VPC creada con ID: {vpc_id}')

print("🔧 Habilitando soporte de DNS para la VPC...")

# Habilitar soporte de DNS
ec2.modify_vpc_attribute(
    VpcId=vpc_id,
    EnableDnsSupport={'Value': True}
)
print("🌐 Soporte de DNS habilitado.")

print("🔧 Habilitando nombres de host DNS para la VPC...")

# Habilitar nombres de host DNS
ec2.modify_vpc_attribute(
    VpcId=vpc_id,
    EnableDnsHostnames={'Value': True}
)
print("🏷️ Nombres de host DNS habilitados.")

print("🛠️ Creando etiqueta para la VPC...")

# Crear una etiqueta para la VPC
ec2.create_tags(
    Resources=[vpc_id],
    Tags=[{'Key': 'Name', 'Value': 'DemoVPC'}]
)
print("🏷️ Etiqueta 'DemoVPC' creada para la VPC.")

print("🎉 Proceso de creación y configuración de la VPC completado.")


🚀 Iniciando la creación de la VPC...
✅ VPC creada con ID: vpc-0524e23d560960cf8
🔧 Habilitando soporte de DNS para la VPC...
🌐 Soporte de DNS habilitado.
🔧 Habilitando nombres de host DNS para la VPC...
🏷️ Nombres de host DNS habilitados.
🛠️ Creando etiqueta para la VPC...
🏷️ Etiqueta 'DemoVPC' creada para la VPC.
🎉 Proceso de creación y configuración de la VPC completado.


## Creación y Configuración de Subnets 🌐🔧
Una VPC se divide en subnets para organizar y gestionar los recursos de red de manera eficiente. A continuación, crearemos una subnet pública con acceso a Internet y una subnet privada sin acceso directo a Internet. También configuraremos las tablas de rutas y un Internet Gateway para habilitar la comunicación adecuada. 🚀


In [11]:
# 🌐 Crear la subnet pública
response_public_subnet = ec2.create_subnet(
    VpcId=vpc_id,
    CidrBlock='10.0.1.0/24',
    AvailabilityZone='us-east-1a'
)
public_subnet_id = response_public_subnet['Subnet']['SubnetId']
print(f'🟢 Subnet pública creada con ID: {public_subnet_id}')

# 🏷️ Crear una etiqueta para la subnet pública
ec2.create_tags(
    Resources=[public_subnet_id],
    Tags=[{'Key': 'Name', 'Value': 'PublicSubnet'}]
)

# 🔒 Crear la subnet privada
response_private_subnet = ec2.create_subnet(
    VpcId=vpc_id,
    CidrBlock='10.0.2.0/24',
    AvailabilityZone='us-east-1a'
)
private_subnet_id = response_private_subnet['Subnet']['SubnetId']
print(f'🔵 Subnet privada creada con ID: {private_subnet_id}')

# 🏷️ Crear una etiqueta para la subnet privada
ec2.create_tags(
    Resources=[private_subnet_id],
    Tags=[{'Key': 'Name', 'Value': 'PrivateSubnet'}]
)

# 🌍 Crear un Internet Gateway
response_igw = ec2.create_internet_gateway()
igw_id = response_igw['InternetGateway']['InternetGatewayId']
print(f'🌐 Internet Gateway creado con ID: {igw_id}')

# 🔗 Adjuntar el Internet Gateway a la VPC
ec2.attach_internet_gateway(
    InternetGatewayId=igw_id,
    VpcId=vpc_id
)

# 🏷️ Crear una etiqueta para el Internet Gateway
ec2.create_tags(
    Resources=[igw_id],
    Tags=[{'Key': 'Name', 'Value': 'DemoIGW'}]
)

# 🗺️ Obtener la tabla de rutas predeterminada
response_route_tables = ec2.describe_route_tables(
    Filters=[
        {
            'Name': 'vpc-id',
            'Values': [vpc_id]
        }
    ]
)
route_table_id = response_route_tables['RouteTables'][0]['RouteTableId']
print(f'🗺️ Tabla de rutas predeterminada con ID: {route_table_id}')

# 🛣️ Crear una ruta en la tabla de rutas predeterminada que apunte al Internet Gateway
ec2.create_route(
    RouteTableId=route_table_id,
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igw_id
)

# 🔗 Asociar la tabla de rutas predeterminada con la subnet pública
ec2.associate_route_table(
    RouteTableId=route_table_id,
    SubnetId=public_subnet_id
)

# 🗺️ Crear una nueva tabla de rutas para la subnet privada
response_private_route_table = ec2.create_route_table(
    VpcId=vpc_id
)
private_route_table_id = response_private_route_table['RouteTable']['RouteTableId']
print(f'🔒 Tabla de rutas privada creada con ID: {private_route_table_id}')

# 🏷️ Crear una etiqueta para la tabla de rutas privada
ec2.create_tags(
    Resources=[private_route_table_id],
    Tags=[{'Key': 'Name', 'Value': 'PrivateRouteTable'}]
)

# 🔗 Asociar la nueva tabla de rutas con la subnet privada
ec2.associate_route_table(
    RouteTableId=private_route_table_id,
    SubnetId=private_subnet_id
)

print('✅ Configuración de VPC completa. La subnet pública tiene acceso a Internet y la subnet privada tiene su propia tabla de rutas.')

🟢 Subnet pública creada con ID: subnet-00b73f022f7583e6c
🔵 Subnet privada creada con ID: subnet-0ba31041411b84a1f
🌐 Internet Gateway creado con ID: igw-09080f3b32d4a0af1
🗺️ Tabla de rutas predeterminada con ID: rtb-0baf3f42b379caa45
🔒 Tabla de rutas privada creada con ID: rtb-0b46030e8365b1ea9
✅ Configuración de VPC completa. La subnet pública tiene acceso a Internet y la subnet privada tiene su propia tabla de rutas.


# Máquinas Virtuales (EC2 en AWS) 💻

Las máquinas virtuales son emulaciones de sistemas informáticos completos. En AWS, se implementan como instancias EC2 (Elastic Compute Cloud).

Características:
- Emula hardware completo 🖥️
- Control total sobre el sistema operativo ⚙️
- Requiere gestión y mantenimiento del sistema operativo 🛠️
- Escalado manual o automatizado, pero no instantáneo 📈
- Facturación por hora/segundo de ejecución ⏲️

## Creación Instancia EC2 AWS Pública 🚀

En esta sección, se explica cómo:

Lanzar una instancia EC2 en AWS 🖥️
Configurar su grupo de seguridad para permitir conexiones SSH usando EC2 Instance Connect 🔒
Ejecutar un script de instalación de Docker en el momento del lanzamiento de la instancia 🐋


### Creación y Configuración de un Grupo de Seguridad para EC2 Instance Connect 🚀

### Variables necesarias

In [12]:
security_group_name = 'SSMAccessSecurityGroup'
iam_role_name = 'EC2SSMRole'
instance_profile_name = 'EC2SSMInstanceProfile'

#### 1. Crear Rol IAM y Perfil de Instancia
Descripción: Esta función crea un rol IAM y un perfil de instancia necesarios para que la instancia EC2 pueda utilizar Systems Manager Session Manager.

In [13]:
def create_iam_role_and_instance_profile():
    iam = boto3.client('iam')
    
    try:
        # Crear el rol IAM
        print("Creando rol IAM... 👤")
        trust_relationship = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {"Service": "ec2.amazonaws.com"},
                    "Action": "sts:AssumeRole"
                }
            ]
        }
        
        iam.create_role(
            RoleName=iam_role_name,
            AssumeRolePolicyDocument=json.dumps(trust_relationship)
        )
        print(f"Rol IAM '{iam_role_name}' creado. ✅")

        # Adjuntar la política AmazonSSMManagedInstanceCore
        print("Adjuntando política AmazonSSMManagedInstanceCore... 📄")
        iam.attach_role_policy(
            RoleName=iam_role_name,
            PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
        )
        print("Política adjuntada. 🔒")

        # Crear el perfil de instancia
        print("Creando perfil de instancia... 📂")
        iam.create_instance_profile(InstanceProfileName=instance_profile_name)
        print(f"Perfil de instancia '{instance_profile_name}' creado. ✅")

        # Añadir el rol al perfil de instancia
        print("Añadiendo rol al perfil de instancia... 🔗")
        iam.add_role_to_instance_profile(
            InstanceProfileName=instance_profile_name,
            RoleName=iam_role_name
        )
        print(f"Rol '{iam_role_name}' añadido al perfil de instancia. 🔗")

        # Esperar a que el perfil de instancia esté disponible
        print("Esperando a que el perfil de instancia esté disponible... ⏳")
        time.sleep(10)

    except ClientError as e:
        if e.response['Error']['Code'] == 'EntityAlreadyExists':
            print("El rol IAM o el perfil de instancia ya existen.")
            
            # Verificar si el rol está asociado al perfil de instancia
            try:
                response = iam.get_instance_profile(InstanceProfileName=instance_profile_name)
                if not response['InstanceProfile']['Roles']:
                    print("El perfil de instancia existe pero no tiene un rol asociado. Asociando rol...")
                    iam.add_role_to_instance_profile(
                        InstanceProfileName=instance_profile_name,
                        RoleName=iam_role_name
                    )
                    print(f"Rol '{iam_role_name}' añadido al perfil de instancia existente. 🔗")
                else:
                    print("El perfil de instancia ya tiene un rol asociado. ✅")
            except ClientError as inner_e:
                print(f"Error al verificar o actualizar el perfil de instancia: {inner_e} ❌")
                return False
        else:
            print(f"Error al crear el rol IAM o el perfil de instancia: {e} ❌")
            return False

    # Verificación final
    try:
        response = iam.get_instance_profile(InstanceProfileName=instance_profile_name)
        if response['InstanceProfile']['Roles']:
            print(f"Verificación final: El perfil de instancia '{instance_profile_name}' tiene el rol '{iam_role_name}' asociado. ✅")
        else:
            print(f"Advertencia: El perfil de instancia '{instance_profile_name}' no tiene roles asociados. ⚠️")
    except ClientError as e:
        print(f"Error al verificar el perfil de instancia: {e} ❌")

    return True

#### 2. Crear Grupo de Seguridad
Descripción: Esta función crea un grupo de seguridad que permite todo el tráfico de salida pero no permite tráfico de entrada, lo cual es suficiente para Session Manager.

In [14]:
def get_or_create_security_group():
    ec2 = boto3.client('ec2')
    try:
        print("Checking for existing security group... 🔍")
        response = ec2.describe_security_groups(
            Filters=[
                {'Name': 'group-name', 'Values': [security_group_name]},
                {'Name': 'vpc-id', 'Values': [vpc_id]}
            ]
        )
        
        if response['SecurityGroups']:
            security_group_id = response['SecurityGroups'][0]['GroupId']
            print(f"Existing security group found with ID: {security_group_id} 🔒")
            return security_group_id
        
        print("Creating new security group... 🛡️")
        security_group = ec2.create_security_group(
            GroupName=security_group_name,
            Description='Security group for Ubuntu with Session Manager access',
            VpcId=vpc_id
        )
        
        security_group_id = security_group['GroupId']

        try:
            print("Authorizing egress rules for the security group... 🔓")
            ec2.authorize_security_group_egress(
                GroupId=security_group_id,
                IpPermissions=[
                    {
                        'IpProtocol': '-1',
                        'FromPort': -1,
                        'ToPort': -1,
                        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                    }
                ]
            )
        except ClientError as e:
            if e.response['Error']['Code'] != 'InvalidPermission.Duplicate':
                raise
            print("Egress rule already exists, continuing... ✅")

        print(f"New security group created with ID: {security_group_id} 🆕")
        return security_group_id
    except ClientError as e:
        print(f"Error handling security group: {e} ❌")
        return None

#### 3. Crear Instancia EC2
Descripción: Esta función crea una instancia EC2 con el rol IAM, perfil de instancia y grupo de seguridad creados anteriormente. También instala Docker en la instancia

In [15]:
# Read the contents of docker_install.sh
with open('scripts/docker_install.sh', 'r') as file:
    docker_install_script = file.read()
    
def create_ubuntu_instance(security_group_id):
    ec2 = boto3.resource('ec2')
    
    try:
        print("Creando instancia Ubuntu... 🖥️")
        instances = ec2.create_instances(
            ImageId='ami-04a81a99f5ec58529',  # Ubuntu en us-east-1
            InstanceType='t2.micro',
            MinCount=1,
            MaxCount=1,
            NetworkInterfaces=[{
                'SubnetId': public_subnet_id,
                'DeviceIndex': 0,
                'AssociatePublicIpAddress': True,
                'Groups': [security_group_id]
            }],
            IamInstanceProfile={'Name': instance_profile_name},
            UserData=f'''#!/bin/bash
                apt-get update
                apt-get install -y snapd
                snap install amazon-ssm-agent --classic
                systemctl enable snap.amazon-ssm-agent.amazon-ssm-agent.service
                systemctl start snap.amazon-ssm-agent.amazon-ssm-agent.service
                # Docker installation script
                {docker_install_script}
            ''',
            TagSpecifications=[
                {
                    'ResourceType': 'instance',
                    'Tags': [
                        {
                            'Key': 'Name',
                            'Value': 'WebApp'
                        },
                    ]
                },
            ]
        )
        
        instance = instances[0]
        print("Esperando a que la instancia se inicie... ⏳")
        instance.wait_until_running()
        instance.reload()
        
        print(f"Instancia Ubuntu creada con ID: {instance.id} 🎉")
        print(f"IP privada: {instance.private_ip_address} 📍")
        print(f"IP pública: {instance.public_ip_address} 🌐")
        return instance
    except ClientError as e:
        print(f"Error al crear la instancia: {e} ❌")
        return None


#### 4. Función Principal
Descripción: Esta función orquesta todo el proceso de creación y muestra un resumen al final.

In [16]:
def main():
    create_iam_role_and_instance_profile()
    security_group_id = get_or_create_security_group()
    instance_id = create_ubuntu_instance(security_group_id)
    
    print(f"""
🎉 ¡Configuración completada!
📝 Resumen:
   - Instancia EC2 ID: {instance_id}
   - Grupo de Seguridad ID: {security_group_id}
   - Rol IAM: {iam_role_name}
   - Perfil de Instancia: {instance_profile_name}

🔐 La instancia está configurada para acceso mediante Session Manager.
🐳 Docker se instalará durante el proceso de inicialización.
📌 Usa la consola de AWS o el AWS CLI para conectarte a la instancia a través de Session Manager.
""")

# Ejecutar el script
main()

Creando rol IAM... 👤
Rol IAM 'EC2SSMRole' creado. ✅
Adjuntando política AmazonSSMManagedInstanceCore... 📄
Política adjuntada. 🔒
Creando perfil de instancia... 📂
El rol IAM o el perfil de instancia ya existen.
El perfil de instancia existe pero no tiene un rol asociado. Asociando rol...
Rol 'EC2SSMRole' añadido al perfil de instancia existente. 🔗
Verificación final: El perfil de instancia 'EC2SSMInstanceProfile' tiene el rol 'EC2SSMRole' asociado. ✅
Checking for existing security group... 🔍
Creating new security group... 🛡️
Authorizing egress rules for the security group... 🔓
Egress rule already exists, continuing... ✅
New security group created with ID: sg-02aa9742fdfac5332 🆕
Creando instancia Ubuntu... 🖥️
Esperando a que la instancia se inicie... ⏳
Instancia Ubuntu creada con ID: i-08abba30d5494b6fc 🎉
IP privada: 10.0.1.210 📍
IP pública: 54.162.79.243 🌐

🎉 ¡Configuración completada!
📝 Resumen:
   - Instancia EC2 ID: ec2.Instance(id='i-08abba30d5494b6fc')
   - Grupo de Seguridad ID: sg

### Lanzamiento de la Instancia EC2 para la Base de Datos 🗄️🚀
En esta sección, crearemos y configuraremos una instancia EC2 que alojará nuestra base de datos. Esta instancia se lanzará en la subnet privada para mayor seguridad.

- 🌐 Vamos a crear una NAT Gateway para que la instancia pueda conectarse a Internet.
- 🔒 La única manera en que se pueda acceder a ella será mediante el Security Group de la instancia web.
- 🐋 Además, tendrá Docker instalado.
- 🐘 También tendrá PostgreSQL instalado.

#### 1. Creación de la NAT Gateway

In [20]:
def create_nat_gateway_and_update_route_table():
    ec2 = boto3.client('ec2')
    
    try:
        # Create an Elastic IP for the NAT Gateway
        print("Creating Elastic IP... 🌐")
        eip_response = ec2.allocate_address(Domain='vpc')
        allocation_id = eip_response['AllocationId']
        print(f"Elastic IP created with Allocation ID: {allocation_id} 📍")

        # Create NAT Gateway in the public subnet
        print("Creating NAT Gateway in the public subnet... 🚀")
        nat_gateway_response = ec2.create_nat_gateway(
            AllocationId=allocation_id,
            SubnetId=public_subnet_id,
        )
        nat_gateway_id = nat_gateway_response['NatGateway']['NatGatewayId']
        print(f"NAT Gateway created with ID: {nat_gateway_id} 🛠️")

        # Wait for the NAT Gateway to become available
        print("Waiting for NAT Gateway to become available... ⏳")
        waiter = ec2.get_waiter('nat_gateway_available')
        waiter.wait(NatGatewayIds=[nat_gateway_id])
        print("NAT Gateway is now available! 🎉")

        # Update the existing route table
        print("Updating the route table... 🛣️")
        ec2.create_route(
            RouteTableId=private_route_table_id,
            DestinationCidrBlock='0.0.0.0/0',
            NatGatewayId=nat_gateway_id
        )
        print(f"Route added to route table {private_route_table_id} 📋")

        # Check if the route table is associated with the private subnet
        print("Checking if the route table is associated with the private subnet... 🔍")
        associations = ec2.describe_route_tables(RouteTableIds=[private_route_table_id])['RouteTables'][0]['Associations']
        is_associated = any(assoc['SubnetId'] == private_subnet_id for assoc in associations)
        
        if not is_associated:
            print("Associating route table with the private subnet... 🔗")
            ec2.associate_route_table(
                RouteTableId=private_route_table_id,
                SubnetId=private_subnet_id
            )
            print(f"Route table {private_route_table_id} associated with private subnet {private_subnet_id} ✅")
        else:
            print(f"Route table {private_route_table_id} is already associated with private subnet {private_subnet_id} ⚙️")

        return True
    except ClientError as e:
        print(f"Error creating NAT Gateway or configuring routes: {e} ❌")
        return False


print("Starting NAT Gateway creation and route configuration... 🚀")
if create_nat_gateway_and_update_route_table():
    print("NAT Gateway created and routes configured successfully. 🎊")
else:
    print("Failed to create NAT Gateway or configure routes. 😢")

Starting NAT Gateway creation and route configuration... 🚀
Creating Elastic IP... 🌐
Elastic IP created with Allocation ID: eipalloc-05013f6525ea0da03 📍
Creating NAT Gateway in the public subnet... 🚀
NAT Gateway created with ID: nat-038535b4517440eab 🛠️
Waiting for NAT Gateway to become available... ⏳
NAT Gateway is now available! 🎉
Updating the route table... 🛣️
Route added to route table rtb-0b46030e8365b1ea9 📋
Checking if the route table is associated with the private subnet... 🔍
Route table rtb-0b46030e8365b1ea9 is already associated with private subnet subnet-0ba31041411b84a1f ⚙️
NAT Gateway created and routes configured successfully. 🎊


In [32]:
SECURITY_GROUP_NAME = 'PrivateInstanceSSMSG'
IAM_ROLE_NAME = 'SSMPrivateInstanceRole'
IAM_INSTANCE_PROFILE_NAME = 'SSMPrivateInstanceProfile'

#### 2. Creación de Security groups

In [33]:
def get_or_create_security_group():
    ec2 = boto3.client('ec2')
    try:
        # Attempt to get the existing security group
        print("Checking for existing security group... 🔍")
        response = ec2.describe_security_groups(
            Filters=[
                {'Name': 'group-name', 'Values': [SECURITY_GROUP_NAME]},
                {'Name': 'vpc-id', 'Values': [vpc_id]}
            ]
        )
        if response['SecurityGroups']:
            security_group_id = response['SecurityGroups'][0]['GroupId']
            print(f"Existing security group '{SECURITY_GROUP_NAME}' found with ID: {security_group_id} 🔒")
        else:
            # If it doesn't exist, create a new one
            print("Creating new security group... 🛡️")
            response = ec2.create_security_group(
                GroupName=SECURITY_GROUP_NAME,
                Description='Security group for private instance with Session Manager access and PostgreSQL',
                VpcId=vpc_id
            )
            security_group_id = response['GroupId']
            print(f"New security group '{SECURITY_GROUP_NAME}' created with ID: {security_group_id} 🆕")

        # Add the egress rule
        try:
            print("Adding egress rule to the security group... 🔓")
            ec2.authorize_security_group_egress(
                GroupId=security_group_id,
                IpPermissions=[
                    {
                        'IpProtocol': '-1',
                        'FromPort': -1,
                        'ToPort': -1,
                        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                    }
                ]
            )
            print("Egress rule added to the security group. ✅")
        except ClientError as e:
            if e.response['Error']['Code'] == 'InvalidPermission.Duplicate':
                print("Egress rule already exists in the security group. ✅")
            else:
                raise

        # Add the PostgreSQL ingress rule (TODO: Ingress from Webapp security group)
        try:
            print("Adding PostgreSQL ingress rule to the security group... 🔓")
            ec2.authorize_security_group_ingress(
                GroupId=security_group_id,
                IpPermissions=[
                    {
                        'IpProtocol': 'tcp',
                        'FromPort': 5432,
                        'ToPort': 5432,
                        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                    }
                ]
            )
            print("PostgreSQL ingress rule added to the security group. ✅")
        except ClientError as e:
            if e.response['Error']['Code'] == 'InvalidPermission.Duplicate':
                print("PostgreSQL ingress rule already exists in the security group. ✅")
            else:
                raise

        return security_group_id
    except ClientError as e:
        print(f"Error handling the security group: {e} ❌")
        return None

#### 3. Creación de IAM Role e Instance Profile

In [34]:
def create_iam_role_and_instance_profile():
    iam = boto3.client('iam')
    
    try:
        print("Creating IAM role... 👤")
        trust_relationship = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {"Service": "ec2.amazonaws.com"},
                    "Action": "sts:AssumeRole"
                }
            ]
        }
        
        iam.create_role(
            RoleName=IAM_ROLE_NAME,
            AssumeRolePolicyDocument=json.dumps(trust_relationship)
        )
        print(f"IAM role '{IAM_ROLE_NAME}' created. ✅")

        print("Attaching AmazonSSMManagedInstanceCore policy to the role... 📄")
        iam.attach_role_policy(
            RoleName=IAM_ROLE_NAME,
            PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
        )
        print("Policy attached to the role. 🔒")

        print("Creating instance profile... 📂")
        iam.create_instance_profile(InstanceProfileName=IAM_INSTANCE_PROFILE_NAME)
        print(f"Instance profile '{IAM_INSTANCE_PROFILE_NAME}' created. ✅")

        print("Adding role to instance profile... 🔗")
        iam.add_role_to_instance_profile(
            InstanceProfileName=IAM_INSTANCE_PROFILE_NAME,
            RoleName=IAM_ROLE_NAME
        )
        print(f"Role '{IAM_ROLE_NAME}' added to instance profile. 🔗")

        return True
    except ClientError as e:
        if e.response['Error']['Code'] == 'EntityAlreadyExists':
            print("IAM role or instance profile already exists. Continuing... ✅")
            return True
        else:
            print(f"Error creating IAM role or instance profile: {e} ❌")
            return False

#### 4. Creacion de instancia privada con docker y postgres

In [35]:
def create_private_instance(security_group_id):
    ec2 = boto3.resource('ec2')
    
    try:
        print("Creating private Ubuntu instance... 🖥️")
        instances = ec2.create_instances(
            ImageId='ami-04a81a99f5ec58529',  # Ubuntu 22.04 LTS
            InstanceType='t2.micro',
            MinCount=1,
            MaxCount=1,
            NetworkInterfaces=[{
                'SubnetId': private_subnet_id,
                'DeviceIndex': 0,
                'AssociatePublicIpAddress': False,
                'Groups': [security_group_id]
            }],
            IamInstanceProfile={'Name': IAM_INSTANCE_PROFILE_NAME},
            UserData=f'''#!/bin/bash
                apt-get update
                apt-get install -y snapd
                snap install amazon-ssm-agent --classic
                systemctl enable snap.amazon-ssm-agent.amazon-ssm-agent.service
                systemctl start snap.amazon-ssm-agent.amazon-ssm-agent.service
                # Docker installation script
                {docker_install_script}
                # Install PostgreSQL with Docker
                docker run --name postgres-db -e POSTGRES_PASSWORD=mysecretpassword -d -p 5432:5432 postgres:13
                # TODO: secret manager for manage pass and configuration
            ''',
            TagSpecifications=[
                {
                    'ResourceType': 'instance',
                    'Tags': [
                        {
                            'Key': 'Name',
                            'Value': 'DataBase'
                        },
                    ]
                },
            ]
        )
        
        instance = instances[0]
        print("Waiting for instance to start... ⏳")
        instance.wait_until_running()
        instance.reload()
        
        print(f"Private instance created with ID: {instance.id} 🎉")
        print(f"Private IP: {instance.private_ip_address} 📍")
        return instance
    except ClientError as e:
        print(f"Error creating the instance: {e} ❌")
        return None

#### 5. Inicia proceso de creación

In [37]:
print("Starting IAM role, security group, and instance setup... 🚀")
if create_iam_role_and_instance_profile():
    security_group_id = get_or_create_security_group()
    if security_group_id:
        instance = create_private_instance(security_group_id)
        if instance:
            print("Private instance created successfully for use with Session Manager. 🎊")
        else:
            print("Failed to create the private instance. 😢")
    else:
        print("Failed to get or create the security group. 😢")
else:
    print("Failed to create IAM role or instance profile. 😢")

Starting IAM role, security group, and instance setup... 🚀
Creating IAM role... 👤
IAM role or instance profile already exists. Continuing... ✅
Checking for existing security group... 🔍
Existing security group 'PrivateInstanceSSMSG' found with ID: sg-06baebce0e551c6d9 🔒
Adding egress rule to the security group... 🔓
Egress rule already exists in the security group. ✅
Adding PostgreSQL ingress rule to the security group... 🔓
PostgreSQL ingress rule already exists in the security group. ✅
Creating private Ubuntu instance... 🖥️
Waiting for instance to start... ⏳
Private instance created with ID: i-0f84044cf749eb40e 🎉
Private IP: 10.0.2.45 📍
Private instance created successfully for use with Session Manager. 🎊


# [WIP] Containers usando Docker y ECR 🐳🚀
En esta sección, exploramos cómo usar contenedores Docker y Amazon Elastic Container Service (ECS) para desplegar y gestionar aplicaciones en AWS.

**Docker:**
- 🐳 **Docker:** Una plataforma para desarrollar, enviar y ejecutar aplicaciones dentro de contenedores ligeros y portátiles.
- 📦 **Imágenes:** Archivos que contienen todo lo necesario para ejecutar una aplicación: código, runtime, bibliotecas y configuraciones.
- 🚀 **Contenedores:** Instancias ejecutables de imágenes Docker que pueden iniciarse y detenerse rápidamente.

**Amazon ECR:**
- 📦 **ECR (Elastic Container Registry):** Un servicio de almacenamiento completamente gestionado para imágenes Docker, que facilita el almacenamiento, la gestión y el despliegue de imágenes de contenedores.
- 🔒 **Seguridad:** Integración con IAM para controlar el acceso a los repositorios de imágenes.
- 🔄 **Automatización:** Soporte para pipelines de CI/CD para automatizar el despliegue de imágenes.


#### 1. Creación de repositorio ECR

In [38]:
import boto3
from botocore.exceptions import ClientError

# Crear un cliente para ECR
ecr_client = boto3.client('ecr')

# Nombre del repositorio ECR
repository_name = 'spend-wise-app'

try:
    # Crear el repositorio ECR
    response = ecr_client.create_repository(
        repositoryName=repository_name,
        imageTagMutability='MUTABLE',
        imageScanningConfiguration={'scanOnPush': True}
    )

    # Extraer y mostrar detalles del repositorio creado
    repository_uri = response['repository']['repositoryUri']
    print(f"Repositorio ECR creado exitosamente: {repository_uri}")

except ClientError as e:
    # Manejo de errores específicos de AWS
    if e.response['Error']['Code'] == 'RepositoryAlreadyExistsException':
        print(f"El repositorio {repository_name} ya existe.")
    else:
        print(f"Error al crear el repositorio ECR: {e}")


Repositorio ECR creado exitosamente: 021891591921.dkr.ecr.us-east-1.amazonaws.com/spend-wise-app


In [1]:
!docker ps

CONTAINER ID   IMAGE           COMMAND       CREATED          STATUS          PORTS                    NAMES
6624f69651a9   demo-notebook   "/start.sh"   25 seconds ago   Up 24 seconds   0.0.0.0:8888->8888/tcp   goofy_ganguly


# [WIP] Serverless (AWS Lambda) ⚡

La computación serverless permite ejecutar código sin provisionar ni administrar servidores. AWS Lambda es el servicio serverless de AWS.

Características:
- No hay que gestionar servidores 🛠️
- Escalado automático e instantáneo 📈
- Facturación por milisegundo de ejecución ⏱️
- Ideal para cargas de trabajo intermitentes 🌐
- Limitado en tiempo de ejecución y recursos ⏳

### [WIP] Ejemplo: Crear una función Lambda

## [WIP] Contenedores (ECS en AWS)

Los contenedores son unidades estándar de software que empaquetan el código y todas sus dependencias.
AWS ECS (Elastic Container Service) es un servicio de orquestación de contenedores.

Características:
- Ligeros y portables
- Aislamiento de aplicaciones
- Rápido despliegue y escalado
- Eficiente uso de recursos
- Ideal para microservicios

### [WIP] Ejemplo: Crear un cluster ECS

In [ ]:
def create_ecs_cluster(cluster_name):
    response = ecs.create_cluster(clusterName=cluster_name)
    return response['cluster']['clusterArn']

cluster_arn = create_ecs_cluster('MiClusterECS')
print(f"Cluster ECS creado con ARN: {cluster_arn}")

## Comparación de servicios

1. IAM:
   - Gestión centralizada de accesos
   - Crucial para la seguridad en la nube
   - Se integra con todos los servicios AWS
   - Sin costo adicional

2. Máquinas Virtuales (EC2):
   - Mayor control y flexibilidad
   - Requiere más gestión
   - Mejor para aplicaciones que necesitan el sistema operativo completo
   - Costo: Puede ser más alto para cargas de trabajo intermitentes

3. Serverless (Lambda):
   - Sin gestión de servidores
   - Escalado automático
   - Ideal para microservicios y funciones específicas
   - Costo: Muy eficiente para cargas de trabajo variables

4. Contenedores (ECS):
   - Portabilidad y consistencia
   - Menos sobrecarga que las VMs
   - Bueno para microservicios y aplicaciones distribuidas
   - Costo: Equilibrio entre VMs y serverless

La elección depende de los requisitos específicos de tu aplicación, 
la experiencia de tu equipo y las necesidades de gestión y escalabilidad.